In [11]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from DockingEnvironment import *
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, ActionNoise

In [34]:
env = DockingEnvironment(radius=2, velthresh=1)

In [7]:
state, info = env.reset()
state, env.x

(array([19.8939932 , 33.82870989, 23.98617155, -0.08705068, -0.26763769,
         0.09916213]),
 array([19.67487132, 33.86218255, 24.28336735, -0.16207567, -0.48456065,
         0.16292168]))

In [8]:
for _ in range(10):
    tot_rew = 0
    i = 0

    state0, info = env.reset()
    while True:
        i += 1
        action = env.action_space.sample()
        state, rew, term, trunc, info = env.step(action)
        tot_rew += rew
        if term:
            print(i, np.linalg.norm(env.x[3:]), tot_rew, rew)
            break

1
34 0.6510494747008642 -0.10628645465765085 -0.10628645465765085
1
192 1.6394251861934832 -0.6349639538999434 -0.6349639538999434
1
273 2.2843408476619693 -0.79744971263881 -0.79744971263881
1
440 2.4432455961836435 -0.6672148696888144 -0.6672148696888144
2
1101 2.3467355739688167 -2 -2
1
770 2.2831801436491883 -0.37373821452914335 -0.37373821452914335
2
643 2.5853921136870976 -2 -2
1
1598 2.0520071505557342 -0.8418190912363215 -0.8418190912363215
1
21 1.0251651687958623 -0.12044660784807071 -0.12044660784807071
1
260 2.815920672591052 -0.4800723214746443 -0.4800723214746443


In [7]:
np.linalg.norm(env.x[:3])

12.739089395490788

In [11]:
5/26.08 -1

-0.808282208588957

In [6]:
env.x

array([-1.48184631e-02,  1.06406261e+01,  7.00437401e+00, -2.09485066e-01,
        1.57126996e-01, -7.49254848e-03])

In [35]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=-.1*np.ones(n_actions), sigma=0.1 * np.ones(n_actions))
#action_noise = OrnsteinUhlenbeckActionNoise(-0.5*np.ones(n_actions), 0.1 * np.ones(n_actions))

model = TD3("MlpPolicy", env, verbose=1, action_noise=action_noise, learning_starts=100000)
model.learn(total_timesteps=600000, log_interval=10)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
1
1
1
1
1
1
1
1
1
1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 116      |
|    ep_rew_mean     | -0.762   |
| time/              |          |
|    episodes        | 10       |
|    fps             | 2287     |
|    time_elapsed    | 0        |
|    total_timesteps | 1155     |
---------------------------------
1
1
1
1
1
1
1
1
1
1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 115      |
|    ep_rew_mean     | -0.819   |
| time/              |          |
|    episodes        | 20       |
|    fps             | 2332     |
|    time_elapsed    | 0        |
|    total_timesteps | 2305     |
---------------------------------
1
1
1
1
1
1
1
1
1
1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 123      |
|    ep_rew_mean     | -0.83    |
| time/              |        

KeyboardInterrupt: 

In [157]:
action_noise._mu = np.zeros(3)
action_noise._sigma = 1e-6 * np.ones(3)

In [164]:
tot_rew = 0
i = 0

state, info = env.reset()
while True:
    i += 1
    action, _ = model.predict(state)
    print(state, action)
    state, rew, term, trunc, info = env.step(action)
    tot_rew += rew
    if term:
        print(i, state, tot_rew)
        break

[32.04952795 44.66032151 37.87090672 -0.73423939 -0.8587518   0.83666124] [-1. -1. -1.]
[31.7906888  44.61749303 37.93504545 -0.9869368  -0.65840731  0.55343033] [-1. -1. -1.]
[32.01898883 44.61982917 37.97286432 -0.96645246 -0.97357846  0.71158505] [-1. -1. -1.]
[31.96080045 44.38151996 38.30791069 -0.83444131 -1.17625093  0.47963855] [-1. -1. -1.]
[31.99293938 44.11597689 38.27795122 -0.98474775 -1.22870853  0.3099565 ] [-1. -1. -1.]
[31.57178065 44.17759421 38.19339025 -1.18021738 -1.3955076   0.14506437] [-1. -1. -1.]
[31.4900033  44.10261559 38.41417472 -1.6668205  -1.54626077  0.3195645 ] [-1. -1. -1.]
[31.49010701 44.10424455 38.54542733 -1.504125   -1.54082315  0.10925467] [-1. -1. -1.]
[31.1650637  43.75654491 38.54267283 -1.44985052 -1.6203509   0.13805024] [-1. -1. -1.]
[31.39076617 43.48878194 37.99110519 -1.70638101 -1.52213619 -0.12488924] [-1. -1. -1.]
[ 3.08411011e+01  4.33875817e+01  3.81103534e+01 -1.83839336e+00
 -1.66144702e+00 -4.14975975e-02] [-1. -1. -1.]
[30.722

In [162]:
results = []
for i in range(1000):
    print(i, end='\r')
    state, info = env.reset()
    j = 0
    while True:
        j += 1
        action, _ = model.predict(state)
        state, rew, term, trunc, info = env.step(action)
        if term:
            results.append(1 if rew == 1 else 0)
            break

3
3
4
3
3
3
3
3
3
3
30
31
32
33
34
35
36
37
38
39
30
31
32
43
34
35
36
37
38
39
30
31
32
33
34
35
36
37
48
39
30
31
32
33
34
35
36
47
38
39
30
31
32
33
34
35
36
37
38
39
40
31
32
33
34
35
36
37
38
39
30
31
32
33
34
45
36
37
38
39
30
31
32
33
34
35
36
47
38
39
30
31
32
33
34
35
36
37
38
39
400
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
427
328
329
330
331
332
333
334
335
336
337
338
339
340
341
442
343
344
345
346
347
348
349
350
351
352
353
354
455
356
357
358
359
360
361
362
363
364
365
366
467
368
469
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
300
301
302
303
304
305
306
307
308
309
310
311
312
413
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
435
336
337
338
339
340
441
342
343
344
345
346
347
448
349
350
351
452
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
37

In [163]:
np.mean(results)

0.945

In [36]:
model.save("./docking2.zip")